<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>CS 144 - Winter 2024 - Mini-Assembler</h1></center>
<center><h1>Due: Sunday, March 17th, 2024 @ 11:59pm</h1></center>

### Enter your information below:

<div style="color: #000000;background-color: #EEEEFF">
    Your Name (submitter):  <br>
    Your student ID (submitter):
<br>
<br>
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
<br>    
<br>
<b>Instruction for submissions:</B> when you have completed this project, download this .ipynb file to your computer by left-clicking on the file name, and submit to <a href="https://elearn.ucr.edu/">Canvas</A> by the deadline. 
<br>
<br>
<B>Late work:</B> There is no late deadline for the final project, except for the most serious circumstances (illness, medical emergency, etc.) which have to documented.
</div>


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Assembler</h1></center>
<br>


The objective of this project is to implement software to assemble a (very short and non-repetitive) genome from synthetic reads. Here we assume that reads have no sequencing errors. (Credit: Ben Langmead)


**Step 1:** Let's get the reads in FASTA format. The reads come from the same synthetic genome and each is 100 bp long. For simplicity, these reads don't have any quality values. 

In [43]:
!wget http://www.cs.ucr.edu/~stelo/cs144winter24/data/reads.fa

--2024-02-14 01:11:22--  http://www.cs.ucr.edu/~stelo/cs144winter24/data/reads.fa
Resolving www.cs.ucr.edu (www.cs.ucr.edu)... 169.235.30.15
Connecting to www.cs.ucr.edu (www.cs.ucr.edu)|169.235.30.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 545000 (532K)
Saving to: 'reads.fa.2'

     0K .......... .......... .......... .......... ..........  9%  395K 1s
    50K .......... .......... .......... .......... .......... 18% 13.1M 1s
   100K .......... .......... .......... .......... .......... 28% 2.62M 0s
   150K .......... .......... .......... .......... .......... 37% 48.3M 0s
   200K .......... .......... .......... .......... .......... 46% 2.86M 0s
   250K .......... .......... .......... .......... .......... 56% 9.47M 0s
   300K .......... .......... .......... .......... .......... 65% 48.3M 0s
   350K .......... .......... .......... .......... .......... 75% 73.9M 0s
   400K .......... .......... .......... .......... .......... 84% 75.3M 0s
   

**Step 2:** Write code to find each read's *best buddy to the right* with a time-efficient algorithm. For each read **A**, find the other read **B** that has the **longest** suffix/prefix exact match with **A**. In other words, if a suffix of **A** matches a prefix of **B**, **B** is **A**'s *best buddy* to the right. **However**, if there is a tie, or if the longest suffix/prefix match is less than 40 nucleotides long, then **A** *has no best buddy to the right*.  For each read, your program should output either (a) nothing, if there is no best buddy to the right, or (b) a single, space-separated line with the IDs of **A** and **B** and the length of the overlap, like this:

    0255/2 2065/1 88

This indicates a 88 bp suffix of the read with ID **0255/2** that matches a prefix of the read with ID **2065/1**.  Because of how we defined best buddy, it also means no other read besides **2065/1** has a prefix of 88bp or longer that is also a suffix of read **0255/2**.  A corrolary of this is that a particular read ID should appear in the first column of your program's output at most once.  Also, since we require the overlap to be at least 40 bases long, the last column should not contain any number smaller than 40.

Notes:
* You can assume all reads are error-free and from the forward strand.  In other words, you do **not** need to consider sequencing errors or reverse complements
* The order of the output lines is not important
* Reading/parsing the FASTA files can be done with BioPython, or by borrowing code from <A HREF="https://nbviewer.jupyter.org/github/BenLangmead/comp-genomics-class/blob/master/notebooks/FASTA.ipynb">this</A> if you want your own custom solution

Important: Try to avoid comparing every read to every other read when looking for suffix/prefix matches. This can be achieved by grouping "similar" reads, that is reads that share common k-mers.  For instance, you can build a dictionary where the keys are k-mers and the values are sets containing the names of all reads containing that k-mer.  Since you are looking for overlaps of length at least 40, you only need to compare reads if they have at least 1 40-mer in common.

You are allowed to use any Python package that you want. You should be able install packages using `!pip install package`. Remember that it is mandatory to acknowledge sources.

In [44]:
k = 40

In [50]:
class ReadObject:


    def __init__(self, name, sequence) -> None:
        
        self.name = name
        self.sequence = sequence

        
        self.bbrName : str = None
        self.bbrSequence : str = None

        self.bbrLength : int = 0

In [51]:
def convertFastaToDict(fastaFilePath : str) -> dict:

        nameToReadObj : dict = {}



        # Get file contents

        with open(fastaFilePath, 'r') as file:



            line1 = file.readline()
        

            while line1:

                line2 = file.readline()
                
            
                if not line2:
                    break 
                

                line1 = line1.strip()
                name = line1[1:]
                sequence =  line2.strip()
                
                
                
                # Add to dictionary

                nameToReadObj[name] = ReadObject(name,sequence)

                line1 = file.readline()
        
        return nameToReadObj




In [52]:
# This function takes in the dictionary of reads
# Using the dictionary or reads, it groups reads with the same 40-mer in either prefix or suffix. 
# Output : dictionary where:
            # key = 40-mer
            # value = set of names of reads that share the same 40-mer


# (Concern: Do we need to separate prefix/suffix?) (Answer: No, unlikely there will be similar suffixes or prefixes, and if so, we'll realize that when we compare the strings.)


k = 40

def storeSimilarReads(reads : dict) -> dict:

    kmerToReadNames : dict[str, set] = dict()


    for name, readObject in reads.items():
        
        sequence = readObject.sequence
        prefix = sequence[:k]
        

        if prefix not in kmerToReadNames:

            kmerToReadNames[prefix] = {name}
        
        else:

            kmerToReadNames[prefix].add(name)
        

        suffix = sequence[-k:]

        if suffix not in kmerToReadNames:
            kmerToReadNames[suffix] = {name}
        
        else:
            kmerToReadNames[suffix].add(name)


    return kmerToReadNames



# reads : dict = convertFastaToDict("reads.fa")

# kmerToReadNames : dict = storeSimilarReads(reads)


# count = 0
# for kmer , setOfNames in kmerToReadNames.items():

#     if(len(setOfNames) > 1):
#         count +=1


# print(count)

# Count should be 3338.




In [53]:

# This function takes in 2 strings and outputs whether or not sequenceToTheRight is a BBR of mainSequence
# If yes, then it returns True, theCommonSequence
# If no, then it returns False, None


def isBBR(mainSequence : str, rightSequence : str, mainSequenceBBRLength: int ):




    mainSequenceTracker = len(mainSequence) - mainSequenceBBRLength
    rightSequenceTracker = mainSequenceBBRLength
    



    # if we get to this point, that means there is a larger BBR for main
    # while we still have characters to read

    commonSequence = mainSequence[-mainSequenceBBRLength:]
    
    while mainSequenceTracker >= 0 and rightSequenceTracker < len(rightSequence):

        # Compare corresponding characters

        if(mainSequence[mainSequenceTracker] == rightSequence[rightSequenceTracker]):

            commonCharacter = mainSequence[mainSequenceTracker]
            commonSequence += commonCharacter

       

            mainSequenceTracker -= 1
            rightSequenceTracker += 1

            


        else:
            break

        #    self.name = name
        # self.sequence = sequence

        
        # self.bbrName : str = None
        # self.bbrSequence : str = None

        # self.bbrLength : int = 0

    
    return commonSequence


In [55]:

import itertools


# Input: Dictionary with the same k-mers grouped

# Function: Compares names with same k-mers and stores the bbr name and sequence info in the ReadObject

# Output: The modified name to ReadObject dictionary

def getBBR(nameToReadObject : dict, kmerToReadNames : dict) -> dict[str, ReadObject]:


    for kmer, setOfNames in kmerToReadNames.items():

        name_pairs = itertools.combinations(setOfNames, 2)

        for pair in name_pairs:
            
            mainSequenceName = pair[0]
            rightSequenceName = pair[1]

            mainSequence = nameToReadObject[mainSequenceName].sequence

            nameToReadObject[mainSequenceName].bbrLength = 40
            mainSequenceBBRLength = nameToReadObject[mainSequenceName].bbrLength

            rightSequence = nameToReadObject[rightSequenceName].sequence

            theCommonSequence = isBBR(mainSequence,rightSequence,mainSequenceBBRLength)

            nameToReadObject[mainSequenceName].bbrName = rightSequenceName
            nameToReadObject[mainSequenceName].bbrSequence = theCommonSequence

            # TODO: Eliminate ties 
            
            nameToReadObject[mainSequenceName].bbrLength = len(theCommonSequence)
            






            mainSequenceName = pair[1]
            rightSequenceName = pair[0]

            mainSequence = nameToReadObject[mainSequenceName].sequence


            nameToReadObject[mainSequenceName].bbrLength = 40
            mainSequenceBBRLength = nameToReadObject[mainSequenceName].bbrLength

            rightSequence = nameToReadObject[rightSequenceName].sequence

            theCommonSequence = isBBR(mainSequence,rightSequence,mainSequenceBBRLength)

            nameToReadObject[mainSequenceName].bbrName = rightSequenceName
            nameToReadObject[mainSequenceName].bbrSequence = theCommonSequence
            nameToReadObject[mainSequenceName].bbrLength = len(theCommonSequence)



            








    return nameToReadObject

nameToReadObject : dict = convertFastaToDict("reads.fa")

kmerToReadNames : dict = storeSimilarReads(nameToReadObject)

nameToReadObject = getBBR(nameToReadObject,kmerToReadNames)

# print(nameToReadObject['0444/1'].bbrLength)

In [56]:
def printOutOverlaps(nameToReadObject : dict):


    for name, readObject in nameToReadObject.items():

        bbrName = readObject.bbrName
        bbrLength = readObject.bbrLength
        print(f"{name} {bbrName} {bbrLength}")




In [57]:
printOutOverlaps(nameToReadObject)

0001/1 None 0
0001/2 2117/2 40
0002/1 2271/2 40
0002/2 1023/2 40
0003/1 0746/2 42
0003/2 2343/1 40
0004/1 0865/1 40
0004/2 None 0
0005/1 0014/2 40
0005/2 0216/1 41
0006/1 0511/1 40
0006/2 0328/2 40
0007/1 1089/2 41
0007/2 1521/2 40
0008/1 0051/2 40
0008/2 0211/1 40
0009/1 1035/1 40
0009/2 2180/2 40
0010/1 2082/2 40
0010/2 None 0
0011/1 0291/1 40
0011/2 None 0
0012/1 0275/2 40
0012/2 0063/1 40
0013/1 0043/2 41
0013/2 0422/2 40
0014/1 1417/1 40
0014/2 0481/1 40
0015/1 1846/2 41
0015/2 None 0
0016/1 0489/1 40
0016/2 0280/1 40
0017/1 2367/2 40
0017/2 2011/2 41
0018/1 0500/1 40
0018/2 None 0
0019/1 0237/1 40
0019/2 2038/1 40
0020/1 1034/2 40
0020/2 1178/1 40
0021/1 None 0
0021/2 2036/2 40
0022/1 0266/2 40
0022/2 0599/2 40
0023/1 2215/1 40
0023/2 0963/1 40
0024/1 0075/2 41
0024/2 1142/1 40
0025/1 0702/1 41
0025/2 0408/2 40
0026/1 0217/2 41
0026/2 2187/1 40
0027/1 0083/2 40
0027/2 0519/2 40
0028/1 1850/2 40
0028/2 1540/2 40
0029/1 1489/2 40
0029/2 2132/2 40
0030/1 0618/1 40
0030/2 2026/2 40
0

**Step 3:** Write a program that takes the output of the overlap program from Step 2 and creates **uni**quely assemblable con**tigs** (unitigs), using the algorithm described below.

We already determined each read's best buddy *to the right*.  I'll abbreviate this as bbr.  We did not attempt to compute each read's best buddy *to the left* (bbl), but we can infer it from the bbrs.  Consider the following output:

    A B 60
    E A 40
    C B 70
    D C 40

**A**'s bbr is **B**.  But **B**'s bbl is **C**, *not* **A**!  Your program should form unitigs by joining together two reads **X** and **Y** if they are *mutual best buddies*.  **X** and **Y** are mutual best buddies if **X**'s bbr is **Y** **and** **Y**'s bbl is **X**, or vice versa.  In this example, we would join **D**, **C**, and **B** into a single unitig (and in that order), and would join reads **E** and **A** into a single unitig (also in that order).

Your program's output should consist of several entries like the following, with one entry per unitig:

    START UNITIG 1 D
      C 40
      B 70
    END UNITIG 1
    START UNITIG 2 E
      A 40
    END UNITIG 2

The first entry represents a unitig with ID **1** consisting of 3 reads.  The first (leftmost) read is **D**.  The second read, **C**, has a **40** nt prefix that is a suffix of the previous read (**D**).  The third (rightmost) read in the contig (**B**) has a **70** bp prefix that is a suffix of the previous read (**C**).

Each read should be contained in exactly one unitig.  The order of unitigs in the file is not important, but the unitig IDs should be integers and assigned in ascending order.

You are allowed to use any Python package that you want but it is mandatory to acknowledge sources.

**Hint:** the correct solution for this particular input consists of exactly 4 unitigs

In [ ]:
## YOUR CODE HERE
## you are allowed to use any python library you want

**Step 4**: Write a program that produces the final genome by processing the unitig file. This requires that you compare the unitigs to each other, think about what order they must go in, and then put them together accordingly. The final genome should be written in FASTA format (again, use BioPython or write your own code). You are allowed to use any Python package that you want, but it is mandatory to acknowledge sources.

**Hints:**

* the correct genome is 7959 nucleotides long
* to learn how the unitigs should go together, try overlapping them with each other
* a unitig can be used more than once in the solution

In [ ]:
## YOUR CODE HERE
## you are allowed to use any python library you want